In [1]:
import pandas as pd
import ast

In [2]:
# Cargar las bases de datos
df_user_items = pd.read_parquet('../Data/df_user_items.parquet')

df_user_reviews = pd.read_parquet('../Data/df_user_reviews.parquet')

df_steam_games = pd.read_parquet('../Data/df_steam_games.parquet')

# Funcion developer

Recibe como parametro "developer" y devuelve la cantidad de items y porcentaje de contenido free por año según empresa desarrolladora.

In [15]:
# Función para obtener el año
def release_year(release_date):
    try:
        return pd.to_datetime(release_date).year
    except (TypeError, ValueError):
        return None


df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})

# Aplicar la función a la columna "release_date" y crear una nueva columna "release_year"
df_steam_games['release_year'] = df_steam_games['release_date'].apply(release_year).astype(pd.Int32Dtype(), errors='ignore')

def developer(desarrollador):
    # Filtrar el DataFrame para obtener solo las filas relacionadas con el developer
    df_developer = df_steam_games[df_steam_games['developer'] == desarrollador]

    # Calcula la cantidad de items por año
    items_per_year = df_developer.groupby(df_developer['release_date'].dt.year)['item_id'].count()
    items_per_year = items_per_year.astype(int)


    # Calcula la cantidad de items gratis por año
    free_items = df_developer[df_developer['price'] == 0].groupby(df_developer['release_date'].dt.year)['item_id'].count()

    # Calcula el porcentaje de items gratis por año
    free_content = (free_items.all() / items_per_year * 100).fillna(0).astype(int)
    
    # Formatea los años en el resultado
    items_per_year = {year: count for year, count in items_per_year.items()}
    free_content = {year: f"{value}%" for year, value in free_content.items()}
    
    result = {
        'Cantidad por año': f"{items_per_year}",
        'Porcentaje gratis por año': f"{free_content}"
    }
    
    return result

In [16]:
developer('Valve')

{'Cantidad por año': '{1998: 1, 1999: 1, 2000: 2, 2001: 1, 2003: 1, 2004: 5, 2006: 2, 2007: 3, 2008: 1, 2009: 1, 2010: 2, 2011: 1, 2012: 1, 2016: 1, 2017: 2}',
 'Porcentaje gratis por año': "{1998: '100%', 1999: '100%', 2000: '50%', 2001: '100%', 2003: '100%', 2004: '20%', 2006: '50%', 2007: '33%', 2008: '100%', 2009: '100%', 2010: '50%', 2011: '100%', 2012: '100%', 2016: '100%', 2017: '50%'}"}

# Funcion userdata

Recibe como parametro "user_id" y devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a "recommend" y cantidad de items.

In [33]:
df_user_items['item_id'] = df_user_items['item_id'].astype(pd.Int32Dtype())
df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(pd.Int32Dtype())
df_steam_games['item_id'] = df_steam_games['item_id'].dropna().astype(pd.Int32Dtype())

def userdata(user_id):
    user_id = str(user_id)
    # Filtrar el DataFrame "user_items" para obtener solo las filas relacionadas con el usuario
    user = df_user_items[df_user_items['user_id'] == user_id]

    # Verificar si el usuario se encuentra en el DataFrame
    if user.empty:
        return pd.DataFrame()

    # Unir "user_items" con "steam_games" para obtener la información de precios
    df_items_merged = user.merge(df_steam_games, on='item_id', how='inner')

    # Calcular la cantidad de dinero gastado por el usuario
    spent_money = df_items_merged['price'].sum()
    spent_money = f"${spent_money:.2f} USD"

    # Unir "user_items" con "user_reviews" para obtener la información de recomendación
    df_reviews_merged = pd.merge(user, df_user_reviews, on='item_id', how='inner')

    # Calcular la cantidad de items y el porcentaje de recomendación en base a "recommend"
    items = len(df_reviews_merged)
    recommend_percentage = df_reviews_merged[df_reviews_merged['user_id_x'] == user_id]['recommend'].mean() * 100

    # Crear un DataFrame con los resultados
    resultados = {
        'Usuario': user_id,
        'Dinero gastado': spent_money,
        '% de recomendacion': recommend_percentage.round(2),
        'Cantidad de items': items,
    }

    return resultados

In [34]:
userdata(76561197970982479)

{'Usuario': '76561197970982479',
 'Dinero gastado': '$9.99 USD',
 '% de recomendacion': 96.97,
 'Cantidad de items': 33}

# Funcion UserForGenre

Recibe "genero" como parametro y devuelve el usuario que acumula mas horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [7]:
df_playtime = df_user_items[['user_id', 'item_id', 'playtime_forever']]

# Crear un DF que contenga las columas "item_id", "genres" y "release_year" y convertir las cadenas de valores en  "genres" a strings
df_genres = df_steam_games[['item_id', 'genres', 'release_year']]

def parse_genres(x):
    try:
        return ast.literal_eval(x) if pd.notna(x) else None
    except (SyntaxError, ValueError):
        return None

df_genres['genres'] = df_genres['genres'].apply(parse_genres)
df_genres = df_genres.explode('genres')

# Unir los DF
df_genres_playtime = df_genres.merge(df_playtime, on='item_id', how='right')

# Crear un DF que contenga las columnas "item_id", "genres" y "release_year" y convertir las cadenas de valores en  "genres" a strings
df_genres = df_steam_games[['item_id', 'genres', 'release_year']]

def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return None

df_genres = df_genres.explode('genres')

df_playtime = df_user_items[['user_id', 'item_id', 'playtime_forever']]

# Unir los DF
df_genres_playtime = df_genres.merge(df_playtime, on='item_id', how='right')

def UserForGenre(genero):

    # Eliminar filas donde el user_id sea un valor nulo
    df = df_genres_playtime.dropna(subset=['user_id'])

    # Filtrar los juegos por el genero dado
    genre = df[df['genres'] == genero]

    # Encontrar al usuario con mas horas jugadas en el genero dado
    user_most_hours = genre.loc[genre['playtime_forever'].idxmax()]['user_id']
    print(user_most_hours)
    # Filtrar el DataFrame para obtener solo las filas relacionadas con el usuario que acumula más horas
    df_user_most_hours = genre.groupby('release_year')['playtime_forever'].sum().reset_index()

    # DF que acumula las horas jugadas por año de lanzamiento
    playtime_years = df_user_most_hours[['release_year', 'playtime_forever']]

    result = {
        'Usuario con mas horas jugadas para el genero dado': user_most_hours,
        'Horas jugadas': [{'Año': int(row['release_year']), 'Horas': int(row['playtime_forever'])} for _, row in playtime_years.iterrows()]
    }
    
    return result

C:\Users\Renato\AppData\Local\Temp\ipykernel_12052\1020732778.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres['genres'] = df_genres['genres'].apply(parse_genres)


In [8]:
UserForGenre('Action')

76561198066412851


{'Usuario con mas horas jugadas para el genero dado': '76561198066412851',
 'Horas jugadas': [{'Año': 1983, 'Horas': 73},
  {'Año': 1984, 'Horas': 12},
  {'Año': 1988, 'Horas': 472},
  {'Año': 1989, 'Horas': 32},
  {'Año': 1990, 'Horas': 316},
  {'Año': 1991, 'Horas': 0},
  {'Año': 1992, 'Horas': 2},
  {'Año': 1993, 'Horas': 96},
  {'Año': 1994, 'Horas': 2305},
  {'Año': 1995, 'Horas': 5042},
  {'Año': 1996, 'Horas': 335},
  {'Año': 1997, 'Horas': 7724},
  {'Año': 1998, 'Horas': 35774},
  {'Año': 1999, 'Horas': 51205},
  {'Año': 2000, 'Horas': 364698},
  {'Año': 2001, 'Horas': 19662},
  {'Año': 2002, 'Horas': 12902},
  {'Año': 2003, 'Horas': 274967},
  {'Año': 2004, 'Horas': 1748160},
  {'Año': 2005, 'Horas': 233599},
  {'Año': 2006, 'Horas': 219166},
  {'Año': 2007, 'Horas': 283628},
  {'Año': 2008, 'Horas': 843072},
  {'Año': 2009, 'Horas': 3807421},
  {'Año': 2010, 'Horas': 4390968},
  {'Año': 2011, 'Horas': 7147805},
  {'Año': 2012, 'Horas': 23955436},
  {'Año': 2013, 'Horas': 1139

# Funcion best_developer_year

Recibe "año" como parametro y devuelve el top 3 de desarrolladores con juegos mas recomendados por usuarios para el año dado.

In [9]:
# Crear un DF que cuente la cantidad de reviews positivas hechas por los usuarios a la desarrolladora
df_merged_reviews = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')
df_merged_reviews = df_merged_reviews.loc[(df_merged_reviews['recommend'] == False) & (df_merged_reviews['sentiment_analysis'] == 0), ['release_year', 'developer']]
df_merged_reviews = df_merged_reviews.groupby(['release_year', 'developer']).size().reset_index(name='reviews')
df_merged_reviews = df_merged_reviews.sort_values(by=['release_year', 'reviews'], ascending=[False, False])
df_merged_reviews = df_merged_reviews.groupby('release_year').head(3)

def best_developer_year(year):
    # Filtrar el DF para obtener las filas relacionadas con el año dado
    top_3 = df_merged_reviews[df_merged_reviews['release_year'] == year]

    # Devolver los resultados de las 3 desarrolladoras con mas reviews positivas
    result = [
        {"Puesto 1": top_3.iloc[0]['developer']},
        {"Puesto 2": top_3.iloc[1]['developer']},
        {"Puesto 3": top_3.iloc[2]['developer']}
        ]
    
    return result

In [10]:
best_developer_year(2012)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Daybreak Game Company'},
 {'Puesto 3': 'Exe Games Inc.'}]

# Funcion developer_reviews_analysis

Recibe a la desarrolladora como parametro y devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [11]:
df_reviews = df_user_reviews [['sentiment_label', 'item_id']]

df_developer = df_steam_games[['developer', 'item_id']]

df_reviews_analysis = df_developer.merge(df_reviews, on= 'item_id', how= 'right')

def developer_reviews_analysis(desarrolladora):
    # Filtrar el DataFrame df_merged_reviews para las reseñas del desarrollador proporcionado.
    developer_reviews = df_reviews_analysis[df_reviews_analysis['developer'] == desarrolladora]

    # Utilizar la función groupby para agrupar por 'sentiment_analysis' y contar el número de registros.
    sentiment_counts = developer_reviews.groupby('sentiment_label').size().reset_index(name='count')

    # Crear un diccionario con el nombre del desarrollador como clave y una lista con la cantidad de reseñas positivas y negativas.
    result = {
        desarrolladora: [
            f"Negative = {sentiment_counts.loc[sentiment_counts['sentiment_label'] == 0, 'count'].values[0]}",
            f"Positive = {sentiment_counts.loc[sentiment_counts['sentiment_label'] == 2, 'count'].values[0]}"
        ]
    }

    return result

In [12]:
developer_reviews_analysis('Valve')

{'Valve': ['Negative = 518', 'Positive = 2621']}